<a href="https://colab.research.google.com/github/abreham-gebremedin/Machine-Learning/blob/main/Text_Classification_with_Na%C3%AFve_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abreham Gebremedin
 (https://abreham-gebremedin.herokuapp.com/)

# **Text Classification with Naïve Bayes**

 


In [1]:
%pylab inline
import IPython
import sklearn as sk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer

Populating the interactive namespace from numpy and matplotlib


In [2]:
news = fetch_20newsgroups(subset='all')

In [3]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
print (type(news.data), type(news.target), type(news.target_names))

<class 'list'> <class 'numpy.ndarray'> <class 'list'>


In [5]:
print(news.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
print(len(news.data))

18846


In [7]:
print(len(news.target))

18846




If we look at, say, the first instance, we will see the content of a newsgroup message, and we can get its corresponding category:


In [8]:


print(news.data[0])
print(news.target[0], news.target_names[news.target[0]])



From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


10 rec.sport.hockey




*Let*'s build the training and testing datasets:


In [9]:


SPLIT_PERC = 0.75
split_size = int(len(news.data)*SPLIT_PERC)
X_train = news.data[:split_size]
X_test = news.data[split_size:]
y_train = news.target[:split_size]
y_test = news.target[split_size:]




# This function will serve to perform and evaluate a cross validation:


In [10]:


def evaluate_cross_validation(clf, X, y, K):
    cv = KFold(K, shuffle=True, random_state=0)

    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print (f"Mean score: {np.mean(scores)} (+/-{sem(scores)})")



In [11]:


clf_1 = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB()),])
clf_2 = Pipeline([('vect', HashingVectorizer(alternate_sign=False)), ('clf', MultinomialNB()),])
clf_3 = Pipeline([('vect', TfidfVectorizer()), ('clf', MultinomialNB()),])



In [12]:


clfs = [clf_1, clf_2, clf_3]
for clf in clfs:
    evaluate_cross_validation(clf, news.data, news.target, 5)



[0.85782493 0.85725657 0.84664367 0.85911382 0.8458477 ]
Mean score: 0.853337340146793 (+/-0.0029134723097208348)
[0.75517241 0.77659857 0.77076148 0.78508888 0.76200584]
Mean score: 0.769925435265917 (+/-0.005270348556491365)
[0.84482759 0.85990979 0.84558238 0.85990979 0.84213319]
Mean score: 0.8504725482840962 (+/-0.003895171759717566)




We will keep the TF-IDF vectorizer but use a different regular expression to pefrom tokenization. The default regular expression: ur"\b\w\w+\b" considers alphanumeric characters and the underscore. Perhaps also considering the slash and the dot could improve the tokenization, and begin considering tokens as Wi-Fi and site.com. The new regular expression could be: ur"\b[a-z0-9-.]+[a-z][a-z0-9-.]+\b". If you have queries about how to define regular expressions, please refer to the Python re module documentation. Let's try our new classifier:


In [17]:
clf_4 = Pipeline([('clf', MultinomialNB()),])



In [18]:
evaluate_cross_validation(clf_4, news.data, news.target, 5)

[nan nan nan nan nan]
Mean score: nan (+/-nan)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py", line 663, in fit
    X, y = self._check_X_y(X, y)
  File "/usr/local/lib/python3.7/dist-packages/sklea